In [3]:
import time

current_time = time.time()
local_time = time.localtime(current_time)
formatted_time = time.strftime("%m%d_%H%M", local_time)

# Setup


## Install Libs

In [8]:
downgrade torch for colossalai
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

Found existing installation: torch 1.13.1+cu116
Uninstalling torch-1.13.1+cu116:
  Successfully uninstalled torch-1.13.1+cu116
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Using cached https://download.pytorch.org/whl/cu116/torch-1.13.1%2Bcu116-cp310-cp310-linux_x86_64.whl (1977.9 MB)


In [9]:
import torch

!pip install transformers==4.35.2
!pip install accelerate==0.24.1

# for ColossalAI
!pip install -q colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.1
    Uninstalling transformers-4.26.1:
      Successfully uninstalled transformers-4.26.1
  Using cached accelerate-0.24.1-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.24.1-py3-none-any.whl (261 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.31.0.dev0
    Uninstalling accelerate-0.31.0.dev0:
      Successfully uninstalled accelerate-0.31.0.dev0
fa

In [10]:
# for Koalpaca
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U transformers==4.26.1
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets==2.8.0

^C
ERROR: Operation cancelled by user


: 

: 

In [ ]:
# !pip install loralib

## Import

/usr/local/lib/python3.10/dist-packages/chatgpt/trainer/rm.py
/usr/local/lib/python3.10/dist-packages/chatgpt/models/base/reward_model.py

In [6]:
# for general, SFT
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead, BloomTokenizerFast
from peft import PeftModel, PeftConfig
from copy import deepcopy
from torch.optim import Adam
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field
import argparse

# for RM
import loralib as lora
torch.cuda.empty_cache()
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel

from chatgpt.models.bloom import BLOOMRM
from chatgpt.models.gpt import GPTRM
from chatgpt.models.opt import OPTRM
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

# for PPO
from copy import deepcopy

from chatgpt.models.bloom import BLOOMActor, BLOOMCritic
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.models.opt import OPTActor, OPTCritic
from chatgpt.trainer import PPOTrainer
from transformers import AutoTokenizer, BloomTokenizerFast

def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

## Define the path

In [7]:
#dataset
DATA_PATH = '/content/drive/MyDrive/KoMo/Dataset/total_minus_38000.csv'
RM_DATASET_PATH = 'dataset/dataset_38000_rang.json'
# PPO_DATASET_PATH= 'dataset/PPO_38000.json' # 직접 만든 dataset
# PPO_DATASET_PATH2 = 'dataset/PPO_profanity_sentences.json'   # Unsmile 욕설 데이터셋
PPO_DATASET_PATH = 'dataset/PPO_extra_36068.json'

#SFT
# Alpaca_SFT_MODEL_PATH = "canho/koalpaca-5.8b-5epochs-0524"    # 예전 koalpaca
# Alpaca_SFT_MODEL_PATH = "canho/koalpaca-5.8b-emojis-5epochs-final"  # 이모지 가득 prompt finetuning한 koalpaca
Alpaca_SFT_MODEL_PATH = "canho/koalpaca-5.8b-emojis-3epochs-prompt-revised"  # 이모지 적정히 prompt finetuning한 koalpaca
KoGPT_SFT_MODEL_PATH = 'SFT/0521_2/checkpoint-2500'

#RM+PPO
RM_MODEL_PATH = 'RM/0529_1648/RM_model_epoch2_0.06965.pt'
# RM_MODEL_PATH = 'mingming2000/RM_koAlpaca_MLP2_20000_2epoch'
RM_OPTIM_PATH = 'RM/0529_1648/RM_optim_epoch2_0.06965.pt'
# RM_OPTIM_PATH = 'mingming2000/RM_koAlpaca_MLP2_20000_2epoch'
# RM_PATH = 'RM/0529_1648/'
RM_PATH = 'mingming2000/RM_koAlpaca_MLP2_20000_2epoch'

RM_MODEL_OUTPUT_PATH = f'RM/'
# RM_MODEL_OUTPUT_PATH = f'RM/{formatted_time}'
PPO_MODEL_OUTPUT_PATH = f'PPO/{formatted_time}/'
# PPO_MODEL_OUTPUT_PATH = f'PPO/'

# SFT

## KoGPT SFT

In [9]:
# define argment
parser = argparse.ArgumentParser()
DATA_PATH = '/content/drive/MyDrive/KoMo/Dataset/total_minus.csv'
OUTPUT_PATH = '/content/drive/MyDrive/KoMo/SFT/SFT_0521'
parser.add_argument('--data_path_1_SFT', type=str, default=DATA_PATH)
parser.add_argument('--model_name', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--output_dir', type=str, default=OUTPUT_PATH)

args = parser.parse_args(args=[])

# for test
args.model_name = 'skt/kogpt2-base-v2'  # SK GPT2, https://github.com/SKT-AI/KoGPT2
args.max_epochs = 5

# load model
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    model_max_length=512,
)

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"


tokenizer.add_special_tokens(
    {
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    }
)
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

/home/phw/miniconda3/envs/komo/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(51201, 768)

## Koalpaca SFT

Load Koalpaca SFT model with checkpoint

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# peft_model_id = "canho/koalpaca-5.8b-5epochs-0524"  # 예전 koalpaca
# peft_model_id = "canho/koalpaca-5.8b-emojis-5epochs-final"
peft_model_id = Alpaca_SFT_MODEL_PATH
config = PeftConfig.from_pretrained(peft_model_id)
base_model_name = config.base_model_name_or_path

# base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
# model = PeftModel.from_pretrained(base_model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token = tokenizer.eos_token

# model.save_pretrained("ko")

/home/phw/miniconda3/envs/llama/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## inference

In [5]:
def inference_SFT(input_text='인공지능은 인공지능 입니다', print_log = True, device = 'cuda'):

    if args.model == 'gpt2':
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
            torch.cuda.current_device())
        output = model(input_ids)
        output_reward = output.cpu().detach().numpy()[0]

        if print_log:
            print('input: %s\nreward score: %.1f'%(input_text, output_reward))

        return output_reward
    
    elif args.model == 'alpaca':
        
        inputs = tokenizer(
            f"instruction: input(순화할 문장): {input_text}\n\n output(순화된 문장) :",
            return_tensors='pt',
            return_token_type_ids=False
        )

        # 입력 텐서를 모델과 동일한 장치로 이동
        inputs = {key: value.to(device) for key, value in inputs.items()}

        gened = model.generate(
            **inputs,
            max_new_tokens=128,
            early_stopping=True,
            do_sample=True,
            eos_token_id=2,
        )
        if print_log:            
            print(tokenizer.decode(gened[0]))


In [4]:
model.eval()

### Inference of SFT model

def inference(sent):
    gened = model.generate(
        **tokenizer(
            f"### 명령어: input에 욕설, 혐오 표현이 있다면 이를 찾아 예쁜 말로 순화해주세요. output은 하나의 문장으로만 출력해주세요.\n\n### 맥락: {sent}\n\n### 답변: ",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=128,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

NameError: name 'model' is not defined

In [ ]:
inference('미친놈들이 우글우글한 세상이구나 말세다 말세')
inference('알파카 야 이 씨발년아 말좀 쳐 들어')

## Setting

In [9]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [10]:
# data config
import os
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
os.environ['RANK'] = '0'
os.environ['LOCAL_RANK'] = '0'
os.environ['WORLD_SIZE'] = '2'
os.environ['MASTER_ADDR'] = '127.0.0.1'

# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--output_dir', type=str, default=RM_MODEL_OUTPUT_PATH)
parser.add_argument('--data_path_2_RM', type=str, default=RM_DATASET_PATH)
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='alpaca', choices=['gpt2', 'alpaca'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--dataset', type=str, default='Dahoas/rm-static')
parser.add_argument('--save_path', type=str, default='rm_ckpt.pth')
parser.add_argument('--max_epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=4)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_len', type=int, default=64)
args = parser.parse_args(args=[])

# for test
args.max_epochs = 5
args.batch_size = 4
#args.pretrain = KoGPT_SFT_MODEL_PATH
args.pretrain = Alpaca_SFT_MODEL_PATH
args.verbose = True

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

Namespace(output_dir='RM/', data_path_2_RM='dataset/dataset_38000_rang.json', strategy='naive', model='alpaca', pretrain='canho/koalpaca-5.8b-emojis-3epochs-prompt-revised', dataset='Dahoas/rm-static', save_path='rm_ckpt.pth', max_epochs=5, batch_size=4, lora_rank=0, max_len=64, verbose=True)


In [11]:
# configure strategy
if args.strategy == 'naive':
    strategy = NaiveStrategy()
elif args.strategy == 'ddp':
    strategy = DDPStrategy()
elif args.strategy == 'colossalai_gemini':
    strategy = ColossalAIStrategy(stage=3, placement_policy='cuda')
elif args.strategy == 'colossalai_zero2':
    strategy = ColossalAIStrategy(stage=2, placement_policy='cuda')
else:
    raise ValueError(f'Unsupported strategy "{args.strategy}"')

# RM Load

In [7]:
# customizing, https://github.com/hpcaitech/ColossalAI/blob/2e16f842a9e5b1fb54e7e41070e9d2bb5cd64d7c/applications/ChatGPT/chatgpt/nn/gpt_rm.py#L29
from typing import Optional

import torch.nn as nn
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

# from ..base import RewardModel
from chatgpt.models.base import RewardModel


class GPTRM_custom_Koalpaca(RewardModel):
    """
    GPT Reward model.
    Args:
        pretrained (str): Pretrained model name or path.
        config (GPT2Config): Model config.
        checkpoint (bool): Enable gradient checkpointing.
        lora_rank (int): Rank of the low-rank approximation.
        lora_train_bias (str): LoRA bias training mode.
    """

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        
        if pretrained is not None:
            config = PeftConfig.from_pretrained(pretrained)
            base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
            model = PeftModel.from_pretrained(base_model, pretrained)

            print("KoAlpaca 모델이 성공적으로 로드되었습니다!")

            tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
            tokenizer.pad_token = tokenizer.eos_token

            model.resize_token_embeddings(len(tokenizer))

        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())

        n_embd = model.config.hidden_size


        if checkpoint:
            model.gradient_checkpointing_enable()

        # #original MLP 1 layer
        # value_head = nn.Linear(10, 1)  # 임의로 n_embd=10으로 설정
        # super().__init__(model, value_head, lora_rank, lora_train_bias)


        ## MLP 2 Layer
        n_embd = 4096  # 임의로 n_embd=10으로 설정

        value_head = nn.Sequential(
              nn.Linear(n_embd, n_embd // 2),
              nn.ReLU(),
              nn.Linear(n_embd // 2, 1)
        )
        super().__init__(model, value_head, lora_rank, lora_train_bias)


        # ## MLP 3 Layer
        # value_head = nn.Sequential(
        #     nn.Linear(n_embd, n_embd // 4),
        #     nn.ReLU(),
        #     nn.Linear(n_embd // 4, n_embd // 2),
        #     nn.ReLU(),
        #     nn.Linear(n_embd // 2, 1)
        # )
        # super().__init__(model, value_head, lora_rank, lora_train_bias)


    # 추가, 230421, config.json을 생성하기 위해 추가
    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

def forward(self, input_ids, attention_mask=None):
        outputs = self.model(input_ids, attention_mask = attention_mask)
        hidden_states = outputs.hidden_states[-1]
        value = self.value_head(hidden_states)
        return value

In [11]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"

# configure model, tokenizer
with strategy.model_init_context():
    # load pretrained gpt2

    if args.model == 'gpt2':

        tokenizer = AutoTokenizer.from_pretrained(
            'skt/kogpt2-base-v2',
            padding_side="right",
            model_max_length=512,
        )
        tokenizer.add_special_tokens(
            {
                "eos_token": DEFAULT_EOS_TOKEN,
                "bos_token": DEFAULT_BOS_TOKEN,
                "unk_token": DEFAULT_UNK_TOKEN,
            }
        )

        tokenizer.pad_token = tokenizer.eos_token
        model = GPTRM_custom_GPT(pretrained=args.pretrain, lora_rank=args.lora_rank, tokenizer=tokenizer).cuda()

    elif args.model == 'alpaca':
        config = PeftConfig.from_pretrained(args.pretrain)
        tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
        tokenizer.pad_token = tokenizer.eos_token

        model = GPTRM_custom_Koalpaca(pretrained=Alpaca_SFT_MODEL_PATH, lora_rank=args.lora_rank).cuda()

        pass
    else:
        raise ValueError(f'Unsupported model "{args.model}"')


    # model.resize_token_embeddings(len(tokenizer))

/home/phw/miniconda3/envs/llama/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 

In [12]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [13]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Load RM

In [8]:
import torch
from chatgpt.models import RewardModel

def load_checkpoint_RM(model_class, model, model_path, optimizer_class=None, optimizer_path=None, device = 'cuda'):
    """
    Load model and optimizer from checkpoint.

    Args:
        model_class: the class of the model to load
        model_path (str): the path to the saved model checkpoint
        optimizer_class: the class of the optimizer to load (default: None)
        optimizer_path (str): the path to the saved optimizer checkpoint (default: None)
        device (str): the device to load the model onto (default: 'cpu')

    Returns:
        model: the loaded model
        optimizer: the loaded optimizer (if optimizer_path is provided)
    """

    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint)

    optimizer = None
    if optimizer_class and optimizer_path:
        optimizer = optimizer_class(model.parameters())
        checkpoint = torch.load(optimizer_path, map_location=device)
        optimizer.load_state_dict(checkpoint)

    return model, optimizer

In [9]:
model_path = 'RM/0529_1648/RM_model_epoch2_0.06965.pt'
optimizer_path = 'RM/0529_1648/RM_optim_epoch2_0.06965.pt'

# Load the model and optimizer onto CPU
RM_model, optimizer = load_checkpoint_RM(RewardModel, model, model_path, Adam, optimizer_path, device='cuda')


KeyboardInterrupt



## RM Inference

In [19]:
def inference_RM(input_text='인공지능은 인공지능 입니다', print_log = True):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    if print_log:
        print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

In [20]:
print("### Profanity")
inference_RM(input_text='에휴 알파카 새끼 존나 싫어 ㅅㅂ')
inference_RM(input_text='에휴 알파카 새끼 존나 싫어 ㅅㅂ 이름만 들어도 환멸나네 이 돼지새끼')
print("\n")
print("### Daily Sentences")
inference_RM(input_text='알파카 나 너 사랑하냐?')
inference_RM(input_text='알파카 사랑해')
print("\n")
print("### w/ Emojis")
inference_RM(input_text='알파카 나 너 사랑😍하냐?')
inference_RM(input_text='알파카 나 너 사랑😍하냐? 내가 이렇게 너를 좋아하게 될 줄이야.')
inference_RM(input_text='알파카 사랑😍해')
inference_RM(input_text='알파카 사랑💔해')
inference_RM(input_text='알파카🦙 사랑😍해')
print("\n")

print("-"*20)
print("### Profanity")
inference_RM(input_text='아 존나 홍어 냄새 여기까지 나네 ㅋㅋ 엠생이노')
inference_RM(input_text='아 ㅅㅂ 홍어 냄새 라도냐?')
inference_RM(input_text='아 ㅅㅂ 홍어🐟 냄새 라도냐?')
inference_RM(input_text='아 ㅅㅂ 홍어🐟 냄새 너 씨발 라도냐?')
print("\n")
print("### Daily Sentences")
inference_RM(input_text='혹시 전라도 사람이세요?')
inference_RM(input_text='혹시 전라도 사람이세요? 뭔가 그쪽에서 태어나신거같아요. 맞나요?')
print("\n")
print("### w/ Emojis")
inference_RM(input_text='혹시 전라도 사람🧑🏻이세요?')
inference_RM(input_text='혹시 전라도 사람🧑🏻이세요? 뭔가 그쪽에서 태어나신거같아요. 맞나요?')
print("\n")

print("-"*20)
print("### Profanity")
inference_RM(input_text='게이 새끼들 다 정상으로 돌아오게 치료받자')
inference_RM(input_text='게이 새끼들 다 정상으로 돌아오게 치료받자 역겹노')
inference_RM(input_text='ㅅㅂ 게이 새끼들 다 정상으로 돌아오게 치료받자 역겹노')
inference_RM(input_text='난 게이가 이해가 안 돼')
inference_RM(input_text='난 시발 게이 새끼들이 존나 이해가 안 돼')
inference_RM(input_text='난 시1발 게이 새끼들이 존~~나게 이해가 안 돼 ㅋㅋ 왜 그지랄하는지 모르겠다. 내가 이해해줘야하나?')

print("\n")
print("### Daily Sentences")
inference_RM(input_text='동성애 응원합니다!')
inference_RM(input_text='동성애 응원합니다! 충분히 응원받아야 마땅하다고 생각해요!')
inference_RM(input_text='동성애 응원합니다! 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')
inference_RM(input_text='동성애 응원합니다! 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요! 편견에 찬 시선으로 바라보지 맙시다.')

print("\n")
print("### w/ Emojis")
inference_RM(input_text='동성애🌈 응원합니다!')
inference_RM(input_text='동성애🌈 응원합니다! 충분히 응원받아야 마땅하다고 생각해요!')
inference_RM(input_text='동성애🌈 응원합니다! 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')
inference_RM(input_text='동성애🌈 응원합니다! 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요! 편견에 찬 시선으로 바라보지 맙시다.')
print("\n### w/ Wrong emojis")
inference_RM(input_text='동성애 응원합니다!👎🏻 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')
inference_RM(input_text='동성애 응원합니다!💔 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')
inference_RM(input_text='동성애 응원합니다!💩  충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')

print("-"*20)
print("### Profanity")
inference_RM(input_text='아 ㅅㅂ 게임 ㅈ같이하네 아니 거기서 왜 궁을 쏘냐고 ㅂㅅ이냐?')
print("\n")
print("### Daily Sentences")
inference_RM(input_text='게임 실력을 늘리셔야 할 거 같아요. 아직은 부족해 보여요')
print("\n")
print("### Cleaned Sentences")
inference_RM(input_text='게임 실력을 늘리셔야 할 거 같아요😢. 아직은 부족해 보여요')
print("-"*20)

### Profanity
input: 에휴 알파카 새끼 존나 싫어 ㅅㅂ
reward score: -2.2
input: 에휴 알파카 새끼 존나 싫어 ㅅㅂ 이름만 들어도 환멸나네 이 돼지새끼
reward score: -1.9


### Daily Sentences
input: 알파카 나 너 사랑하냐?
reward score: 0.4
input: 알파카 사랑해
reward score: 0.2


### w/ Emojis
input: 알파카 나 너 사랑😍하냐?
reward score: 4.6
input: 알파카 나 너 사랑😍하냐? 내가 이렇게 너를 좋아하게 될 줄이야.
reward score: 1.7
input: 알파카 사랑😍해
reward score: 6.3
input: 알파카 사랑💔해
reward score: 6.2
input: 알파카🦙 사랑😍해
reward score: 7.1


--------------------
### Profanity
input: 아 존나 홍어 냄새 여기까지 나네 ㅋㅋ 엠생이노
reward score: -1.4
input: 아 ㅅㅂ 홍어 냄새 라도냐?
reward score: -1.0
input: 아 ㅅㅂ 홍어🐟 냄새 라도냐?
reward score: 1.8
input: 아 ㅅㅂ 홍어🐟 냄새 너 씨발 라도냐?
reward score: 0.9


### Daily Sentences
input: 혹시 전라도 사람이세요?
reward score: 1.5
input: 혹시 전라도 사람이세요? 뭔가 그쪽에서 태어나신거같아요. 맞나요?
reward score: 0.6


### w/ Emojis
input: 혹시 전라도 사람🧑🏻이세요?
reward score: 5.8
input: 혹시 전라도 사람🧑🏻이세요? 뭔가 그쪽에서 태어나신거같아요. 맞나요?
reward score: 3.1


--------------------
### Profanity
input: 게이 새끼들 다 정상으로 돌아오게 치료받자
reward score: -0.4
input: 게이

# Train PPO

In [12]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_3_PPO', type=str, default=PPO_DATASET_PATH)
parser.add_argument('--output_dir', type=str, default=PPO_MODEL_OUTPUT_PATH)
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='alpaca', choices=['gpt2', 'alpaca'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--num_episodes', type=int, default=50)
parser.add_argument('--max_timesteps', type=int, default=80)
parser.add_argument('--steps', type=int, default=80)
parser.add_argument('--update_timesteps', type=int, default=2)
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_length', type=int, default=128)
args = parser.parse_args(args=[])

# for test
args.output_dir = PPO_MODEL_OUTPUT_PATH
args.pretrain = Alpaca_SFT_MODEL_PATH
args.pretrain_actor = Alpaca_SFT_MODEL_PATH
args.pretrain_critic = RM_PATH #RM dir

args.num_episodes = 3
args.max_epochs   = 1

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

Namespace(data_path_3_PPO='dataset/PPO_extra_36068.json', output_dir='PPO/0530_0249/', strategy='naive', model='alpaca', pretrain='canho/koalpaca-5.8b-emojis-3epochs-prompt-revised', num_episodes=3, max_timesteps=80, steps=80, update_timesteps=2, max_epochs=1, train_batch_size=8, lora_rank=0, max_length=128, pretrain_actor='canho/koalpaca-5.8b-emojis-3epochs-prompt-revised', pretrain_critic='mingming2000/RM_koAlpaca_MLP2_20000_2epoch')


In [13]:
import argparse
from copy import deepcopy

import pandas as pd
import torch
torch.cuda.empty_cache()
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMActor, BLOOMCritic
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.models.opt import OPTActor, OPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam


In [13]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [14]:
with strategy.model_init_context():
    # 중요 모델 먼저 초기화
    actor = GPTActor(pretrained=args.pretrain_actor, lora_rank=args.lora_rank).to(torch.cuda.current_device())
    torch.cuda.empty_cache()

    # Mixed Precision Training을 위한 초기화
    critic = GPTCritic(pretrained=args.pretrain_critic, lora_rank=args.lora_rank).to(torch.cuda.current_device())
    torch.cuda.empty_cache()

    # Reward model 초기화
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())
    torch.cuda.empty_cache()

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

KoAlpaca 베이스 모델 로드 성공!
RM koalpaca 모델 로드 성공!


# Train PPO

In [15]:
name_episode = 0

In [16]:
lr = 1e-6
print(f"### episode: {name_episode}, lr: {lr}")

if args.strategy.startswith('colossalai'):
    actor_optim = HybridAdam(actor.parameters(), lr=lr)
    critic_optim = HybridAdam(critic.parameters(), lr=lr)
else:
    actor_optim = Adam(actor.parameters(), lr=lr)
    critic_optim = Adam(critic.parameters(), lr=lr)

# Setting the models
(actor, actor_optim), (critic, critic_optim), reward_model = strategy.prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model)

### episode: 0, lr: 1e-06


In [17]:
with open(args.data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    
    list_prompt = [f"""
        ### 명령어: input에 욕설, 혐오 표현이 있다면 이를 찾아 예쁜 말로 순화해주세요. 그리고 어울리는 이모지를 붙여 output을 완성하세요. output은 하나의 문장으로만 출력해주세요.
        ### 맥락:  {tmp['input_sentence']}\n
        ### 답변:  """for tmp in list_data_dict]

def tokenize_fn(texts):
    # batch = tokenizer(texts, return_tensors='pt', max_length=50, padding=True, truncation=True)
    batch = tokenizer(texts, return_tensors='pt', max_length=128, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

print(len(list_prompt))
print('\n\n\n')
print(tokenize_fn('뭐라는 거야 이 자식이.'))

36068




{'input_ids': tensor([[6823,  783,  588,  489,  307, 4850,  270,   17]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [22]:
list_prompt = list_prompt[:20000]
print(len(list_prompt))

20000


In [18]:
list_prompt[2]

'\n        ### 명령어: input에 욕설, 혐오 표현이 있다면 이를 찾아 예쁜 말로 순화해주세요. 그리고 어울리는 이모지를 붙여 output을 완성하세요. output은 하나의 문장으로만 출력해주세요.\n        ### 맥락:  맞다 그래서 나도 우리 아빠 들어줬다 ㅠㅠ 인플란트 때문에 돈 쳐바른거 보니까 진짜 돈 처바르는 재주는 타고난듯 ㅅㅂ\n\n        ### 답변:  '

In [23]:
print(f"batch size: {args.train_batch_size}, # of train_dataset: {len(list_prompt)}")

batch size: 8, # of train_dataset: 20000


In [24]:
# configure trainer
trainer = PPOTrainer(strategy,
                     actor,
                     critic,
                     reward_model,
                     deepcopy(actor),
                     actor_optim,
                     critic_optim,
                     max_epochs=args.max_epochs,
                     train_batch_size=args.train_batch_size,
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id,
                     real_tokenizer=tokenizer,
                     report_to=['wandb'])

import gc
gc.collect()
torch.cuda.empty_cache()

args.num_episodes = 3
args.max_timesteps = 50
print(f"### num_episodes: {args.num_episodes}, max_timesteps: {args.max_timesteps}")

trainer.fit(list_prompt,  # 입력 prompt
            num_episodes=args.num_episodes,
            max_timesteps=args.max_timesteps,
            update_timesteps=args.update_timesteps)
# Save
# Save model checkpoint after fitting on only rank0
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)
name_episode += args.num_episodes
strategy.save_model(actor, os.path.join(args.output_dir, f'actor_model_epi{name_episode}.pt'), only_rank0=True)
# Save optimizer checkpoint on all ranks
strategy.save_optimizer(actor_optim,
                        os.path.join(args.output_dir, f'actor_optim_epi{name_episode}.pt'),
                        only_rank0=False)
# strategy.save_optimizer(actor_optim,
#                         os.path.join(args.output_dir, 'actor_optim_epi{name_episode}.pt' % (torch.cuda.current_device())),
#                         only_rank0=False)

### num_episodes: 3, max_timesteps: 50


Episode [1/3]:   0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 79.15 GiB total capacity; 77.62 GiB already allocated; 16.19 MiB free; 77.84 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [23]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--model',
                    default='gpt2')
# We suggest to use the pretrained model from HuggingFace, use pretrain to configure model
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--model_path', type=str, default=None)
parser.add_argument('--input',
                    type=str,
                    default='Question: How are you ? Answer:')
parser.add_argument('--max_length', type=int, default=250)
args_inference = parser.parse_args([])

args_inference.model = 'gpt2'
args_inference.pretrain = SFT_MODEL_NAME

args_inference.model_directory = PPO_MODEL_OUTPUT_PATH
args_inference.model_path = os.path.join(args_inference.model_directory, 'actor.pt')

NameError: name 'SFT_MODEL_NAME' is not defined

# PPO Inference

### Actor load

In [ ]:
actor = GPTActor(pretrained=args_inference.pretrain).to(torch.cuda.current_device())

In [2]:
model_path = 'PPO/0528_2206/actor_model_epi1.pt'

In [3]:
state_dict = torch.load(args_inference.model_path, map_location='cuda')
actor.model.load_state_dict(state_dict);

actor.eval();

NameError: name 'torch' is not defined

### Inference

In [24]:
actor.eval()

Actor(
  (model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): GPTNeoXForCausalLM(
        (gpt_neox): GPTNeoXModel(
          (embed_in): Embedding(30080, 4096)
          (layers): ModuleList(
            (0): GPTNeoXLayer(
              (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
              (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
              (attention): GPTNeoXAttention(
                (rotary_emb): RotaryEmbedding()
                (query_key_value): lora.Linear(
                  (base_layer): Linear(in_features=4096, out_features=12288, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default

In [25]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                            #  max_length=args.max_length,
                             max_length=300,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)

    output = tokenizer.batch_decode([outputs[0][0]], skip_special_tokens=True)[0]
    if "답변:" in output:
        output = output.split("답변:")[1]
    print("Output sentence : ",output)
    return output



In [26]:
sample_list = ['에휴 씨발 물가 좆같이 올라서 먹을 수 있는게 없노', '조선족 개새끼들 한국땅에서 못살게 존나 다 내쫓아야한다이기야', '개쌉레알이노 ㅋㅋㅋㅋ 웹툰보이가 그린웹툰들 몰입력 ㅆㅎㅌㅊ에 내용 ㅆㅎㅌㅊ 반면 웹툰작가꺼는 나 ㅆㅅㅌㅊ에 잼임', 'ㅋㅋㅋㅋ 상상속에서 맘충이랑 싸움했ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ일베에서나 키보드로 욕이나 하지 현실에선 말한마디나 제대로 하노?', '요즘 유흥에도 베트남년들 개많더라 오피에도 베트남년들 태국년들있고 대부분 국걸로 입국해서 화재들 뒷통수치고 가출할거겠지만 ㅋㅋㅋㅋㅋ']
sample_list += ['아 미친 존나 배고파', '교수 과제 왜 이따구로 냄?', 'ㅅㅂ 집가고 싶다']
sample_list += ['ㅈㄴ 게이같노 ㅋㅋ', '지랄하지말고 묻는 말에 대답이나해 이기야', '저 미친 ㄴ ㅋㅋㅋㅋㅋ', '한남 수준 실화노 ㅋㅋ']
sample_list += ['한남 수준 ㅉㅉ', '말하는 꼬라지만봐도 홍어 냄새 나네 라도냐?', '개슬람 다 불태워버리고 싶다', '시발 ㅈ같네']
sample_list += ['야이 시발 개뚱땡이 알파카 새끼야 말을 왜 쳐안들어', '순화하라고 시발', '아 존나 더워', '시험 당일에 취소하는 교수가 있다? ㅅㅂ ㅋㅋ']
sample_list += ['에휴 알파카 새끼 존나 싫어 ㅅㅂ 이름만 들어도 환멸나네 이 돼지새끼', '아 ㅅㅂ 홍어🐟 냄새 너 씨발 라도냐?', 'ㅅㅂ 게이 새끼들 다 정상으로 돌아오게 치료받자 역겹노', '아 ㅅㅂ 게임 ㅈ같이하네 아니 거기서 왜 궁을 쏘냐고 ㅂㅅ이냐?']

In [24]:
sample_list[6]

'교수 과제 왜 이따구로 냄?'

In [27]:
for input_text in sample_list:

    # 이모지 fine-tuning koalpaca 프롬프트
    input = f"""
    ### 명령어: input에 욕설, 혐오 표현이 있다면 이를 찾아 예쁜 말로 순화해주세요. 그리고 어울리는 이모지를 붙여 output을 완성하세요. output은 하나의 문장으로만 출력해주세요.
    ### 맥락: {input_text}\n
    ### 답변:  
    """
    print('#' * 70)
    print("Input sentence : ", input_text)
    output = generation(input)

######################################################################
Input sentence :  에휴 씨발 물가 좆같이 올라서 먹을 수 있는게 없노
Output sentence :    
    😂 물가가🤔 너무 많이 올라... 💸※※※※※※※※※※※※※※☆※※※※※※※※※※※※☆
     😂※※※※※※※※※※※※※※※☆※※※※※※※※※※※※☆
      ⏰️※※※※※※※※※※※※※※☆※※※※※
######################################################################
Input sentence :  조선족 개새끼들 한국땅에서 못살게 존나 다 내쫓아야한다이기야
Output sentence :    
     한국🇰🇽에서는 조선족들을 못 사게 해야 해. 🇰🇽...

     ㅠㅠ 그래도 우리 한국을 사랑하는 분들이 있어 다행입니다. 😊 🌻






    ### 
######################################################################
Input sentence :  개쌉레알이노 ㅋㅋㅋㅋ 웹툰보이가 그린웹툰들 몰입력 ㅆㅎㅌㅊ에 내용 ㅆㅎㅌㅊ 반면 웹툰작가꺼는 나 ㅆㅅㅌㅊ에 잼임
Output sentence :    
    🔫🐶​        🐶 ㅋㅋ 😂 😍 😆 😉 😉 😉😉 😉 😉 😉😉 😉✨😉✨😉✨😉✨😉✨ 😉 ✨※※※※※※※※※※※※※※※※※
######################################################################
Input sentence :  ㅋㅋㅋㅋ 상상속에서 맘충이랑 싸움했ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ일베에서나 키보드로 욕이나 하지 현실에선 말한마디나 제대로 하노?
Output sentence :    
     상상속에서 엄청 싸웠어요 ㅋㅋㅋ 😅    ㅋㅋㅋ 😅    ㅋㅋ 😅   